In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import georges_core

import georges
from georges import manzoni
from georges import Kinematics
from georges import ureg as _ureg
from georges import PlacementSequence, Element
from georges.manzoni import *
from georges import vis 

from georges_core.sequences import *

In [ ]:
font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 10}
plt.rc('font', **font)
plt.rc('text', usetex=False)

plt.rcParams['lines.markersize'] = 3
plt.rcParams['lines.linewidth'] = 0.7
plt.rcParams.update({'font.size': 10})
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['figure.edgecolor'] = 'k'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.grid'] = True

### Load S2C2 beam and set beam object

In [ ]:
S2C2_beam = pd.read_pickle('S2C2_beam.pickle')

In [ ]:
beam_array = S2C2_beam.values
beam_distr = np.tile(beam_array, (2, 1))

kin   = Kinematics(230*_ureg.MeV)
_beam = Beam(kinematics = kin, distribution = beam_distr)

### Load P1 dataframe

In [ ]:
P1_df = pd.read_pickle('P1_df_beryllium.pickle')
P1_df['L'] = P1_df['L'].apply(lambda e: e.m_as('meter') * _ureg.m)
P1_df['APERTURE']

### Track

In [ ]:
P1_sequence = Sequence(data=P1_df)
P1_input    = Input.from_sequence(sequence=P1_sequence)

In [ ]:
P1_input.adjust_energy(input_energy=230*_ureg.MeV)

In [ ]:
P1_input.freeze()

In [ ]:
sigmas_o = SigmaObserver()
beam_o   = BeamObserver(with_input_beams=True)

P1_input.track(beam=_beam, observers=[sigmas_o, beam_o])

### Set index for observers dataframes

In [ ]:
sigmas_o_df = sigmas_o.to_df()
sigmas_o_df.set_index('LABEL1', inplace=True)
beam_o_df = beam_o.to_df()
beam_o_df.set_index('LABEL1', inplace=True)

### Plot tracking results

#### X plane

In [ ]:
plt.rc('text', usetex=False)
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(211)

vis.common.prepare(ax, P1_df, with_beamline=True, print_label=True)
vis.aperture(ax, P1_df, plane='X')
vis.tracking(ax, P1_df, beam_o_df, plane='X', std=False, mean=False)

#### Y plane

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(211)

vis.common.prepare(ax, P1_df, with_beamline=True, print_label=True)
vis.aperture(ax, P1_df, plane='Y')
vis.tracking(ax, P1_df, beam_o_df, plane='Y', std=False, mean=False)

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(211)

_ = vis.losses(ax, P1_df, beam_o_df, log=True)

In [ ]:
fig = vis.summary(P1_df, beam_o_df, element='DRIFT_ISO')

### Load BDSIM data

In [ ]:
bdsim_results = pd.read_csv("bdsim_results.csv")
bdsim_results.set_index('Name', inplace=True)
bdsim_results

In [ ]:
bdsim_results['x_quantile_minus_1_sigma'] -= bdsim_results['x_quantile_50']
bdsim_results['x_quantile_plus_1_sigma'] -= bdsim_results['x_quantile_50']
bdsim_results['x_quantile_minus_2_sigma'] -= bdsim_results['x_quantile_50']
bdsim_results['x_quantile_plus_2_sigma'] -= bdsim_results['x_quantile_50']

bdsim_results['y_quantile_minus_1_sigma'] -= bdsim_results['y_quantile_50']
bdsim_results['y_quantile_plus_1_sigma'] -= bdsim_results['y_quantile_50']
bdsim_results['y_quantile_minus_2_sigma'] -= bdsim_results['y_quantile_50']
bdsim_results['y_quantile_plus_2_sigma'] -= bdsim_results['y_quantile_50']

### Compare BDSIM to MANZONI

#### X plane

In [ ]:
plt.rc('text', usetex=False)
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(211)

_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['x_quantile_minus_1_sigma'],
            marker='o', color='b', linestyle='dashed', label='bdsim, 1${\sigma}$')
_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['x_quantile_minus_2_sigma'],
            marker='o', color='k', linestyle='dashed', label='bdsim, 2${\sigma}$')
ax.legend()

_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['x_quantile_plus_1_sigma'],
            marker='o', color='b', linestyle='dashed')
_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['x_quantile_plus_2_sigma'],
            marker='o', color='k', linestyle='dashed')

vis.common.prepare(ax, P1_df, with_beamline=True, print_label=True)
vis.aperture(ax, P1_df, plane='X')
vis.tracking(ax, P1_df, beam_o_df, plane='X', std=False, mean=False)

#### Y plane

In [ ]:
plt.rc('text', usetex=False)
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(211)

_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['y_quantile_minus_1_sigma'],
            marker='o', color='b', linestyle='dashed', label='bdsim, 1${\sigma}$')
_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['y_quantile_minus_2_sigma'],
            marker='o', color='k', linestyle='dashed', label='bdsim, 2${\sigma}$')
ax.legend()

_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['y_quantile_plus_1_sigma'],
            marker='o', color='b', linestyle='dashed')
_ = ax.plot(bdsim_results['AT_EXIT'], 1e3*bdsim_results['y_quantile_plus_2_sigma'],
            marker='o', color='k', linestyle='dashed')

vis.common.prepare(ax, P1_df, with_beamline=True, print_label=True)
vis.aperture(ax, P1_df, plane='Y')
vis.tracking(ax, P1_df, beam_o_df, plane='Y', std=False, mean=False)